## Введение

Перед началом анализа мы загрузим **данные о ледниках из базы WGMS** и выполним их первичную обработку.
Нас интересует, **в каких странах находится больше всего ледников, максимальную васоту ледника**, а также **сколько сантиметров в год теряют ледники в стране, куда отправится белка**.

Все эти расчеты помогут нашей белке 🐿️ понять, на каком леднике лед 🧊 прочнее, а значит - куда безопаснее спрятать желудь, чтобы не случился ☠️*апокалипсис*☠️


Откроем наши датасетики 🧊🐿️

In [84]:
with open('change_band.csv', 'r', encoding='utf-8') as file:
    change_data = file.readlines()

print('Первые 5 строк файла change_band.csv:')
for line in change_data[:5]:
    print(line.strip())

print('\nКоличество строк в файле change_band.csv:', len(change_data)

Первые 5 строк файла change_band.csv:
country,glacier_name,glacier_id,change_id,lower_elevation,upper_elevation,area,elevation_change,elevation_change_unc,volume_change,volume_change_unc,remarks
AR,MARTIAL,917,745,780,800,,,,,,
AR,MARTIAL,917,745,800,900,,,,,,
AR,MARTIAL,917,745,900,1000,,,,,,
AR,MARTIAL,917,745,1000,1100,,,,,,

Количество строк в файле change_band.csv: 4009


## Теперь посмотрим сколько ледников в какой стране есть ⛰️

Начнем обрабатывать файл сразу, так как данные достаточно чистые, единственное нужно проверять на пропуски

In [95]:
filename = 'change_band.csv'

countries_dict = {}

with open(filename, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()

        if not line:
            continue

        parts = line.split(',')

        if len(parts) < 3:  # Check that the string has at least 3 parts (country, name, ID)
            continue

        # Take the country (first part) and the glacier ID (third part)
        country = parts[0]
        glacier_id_str = parts[2]

        # Check that the ID string and country are not empty
        # Check that the ID is a number
        # And check if the string consists only of digits
        if glacier_id_str.isdigit() and country:
            if country not in countries_dict: # If the country is not yet in the dictionary,create an empty set
                countries_dict[country] = set()

            countries_dict[country].add(glacier_id_str)  # Adding the glacier ID to the set for this country

print('Количество уникальных ледников по странам:')
for country in sorted(countries_dict.keys()):
    count = len(countries_dict[country])
    print(f'{country}: {count}')

Количество уникальных ледников по странам:
AR: 2
AT: 13
BO: 2
CA: 10
CH: 6
CO: 2
DE: 5
ES: 1
IN: 2
IS: 1
IT: 4
KE: 1
KG: 11
KZ: 3
NP: 2
PK: 7
RU: 1
SE: 1
SJ: 1
TJ: 1
US: 4


## Выведем страну с наибольшим количеством ледников 🧊
Чекаем самым простым способом: через цикл

In [96]:
max_country = ''
max_count = 0
    # Check all the countries and their number of dictionary
for country in countries_dict:
    count = len(countries_dict[country])
 
    if count > max_count:
        max_count = count
        max_country = country

print(f'Страна с наибольшим количеством ледников: {max_country} ({max_count})')

Страна с наибольшим количеством ледников: AT (13)


## Теперь узнаем максимальную высоту, чтобы у белки был долгий и крутой подъем 🧗🏿‍♀️
Белка-альпинист 

In [106]:
max_height = 0
max_country = ''
max_glacier = ''

with open(filename, 'r', encoding='utf-8') as file:
    header = file.readline() # Read first line and another lines one by one
    for line in file:
        line = line.strip()
        if not line:
            continue  # Skip empty

        parts = line.split(',')

        # Check that the string contains the required number of elements
        if len(parts) < 6:
            continue

        country = parts[0]
        glacier_name = parts[1]
        upper_elevation = parts[5]  # Еake the sixth element [5 index], since the data on the maximum height is in the 6th column

        # If there is a height
        if upper_elevation != '':
            height = float(upper_elevation)

            # If found heigher glacier
            if height > max_height:
                max_height = height
                max_country = country
                max_glacier = glacier_name

print('Максимальная высота ледника:', max_height)
print('Страна:', max_country)
print('Название ледника:', max_glacier)

Максимальная высота ледника: 6500.0
Страна: IN
Название ледника: BARA SHIGRI


## ОКАК. Белочка летит в Индию... Наверное. 🍛

Мы нашли максимальную высоту ледника, но теперь нам нужно проверить, а не слишком ли быстро теряют свою массу ледники в Индии. Для этого посмотрим на индекс изменения высоты ледников (см/год).

In [107]:
india_elev_changes = []

with open(filename, 'r', encoding='utf-8') as file:
    header = file.readline().strip().split(',')

    country_index = header.index('country')
    elev_index = header.index('elevation_change')

    for line in file:
        parts = line.strip().split(',')

        if parts[country_index] == 'IN':
            elev_value = parts[elev_index]

            # Check if empty or not
            if elev_value != '':
                india_elev_changes.append(float(elev_value))

# sort from largest to smallest
india_elev_changes.sort(reverse=True)

print('Значения изменения высот для ледников Индии:')
for value in india_elev_changes:
    print(value)

Значения изменения высот для ледников Индии:
2.882
1.455
0.928
0.633
0.617
0.385
-0.246
-0.608
-0.95
-1.224
-1.622
-1.84
-1.919
-2.348
-2.642
-3.024
-3.065
-3.114
-3.544
-3.631
-3.718
-3.812
-3.964
-4.014
-4.022
-4.102
-4.352
-4.555
-5.5
-5.526
-6.641
-6.7
-7.629
-8.247
-8.513
-8.979
-9.139
-9.141
-9.767
-9.799
-10.008
-10.633
-10.929
-11.148
-11.905


Белка поедет в Индию, потому что ледники там смогут уберечь ее желудь, ведь они расстают еще не скоро...

## Но нам нужно проверить, может ли она привести вагон и маленькую тележку желудей, ведь белка очень любит запасаться на зиму 🛒 (на целый Ледниковый период)

Посчитаем высоту ледника, а затем общую площадь

In [122]:
uppers = []
lowers = []

with open(filename, 'r', encoding='utf-8') as file:
    header = file.readline().strip().split(',')

    name_index = header.index('glacier_name')
    lower_index = header.index('lower_elevation')
    upper_index = header.index('upper_elevation')

    for line in file:
        parts = line.strip().split(',')

        # looking for lines with a glacier bara shigri
        if 'bara shigri' in parts[name_index].lower():
            lower = parts[lower_index]
            upper = parts[upper_index]

            # Check empty or not
            if lower != '' and upper != '':
                lowers.append(float(lower))
                uppers.append(float(upper))

# Сalculate the total height
if lowers and uppers:
    total_height = max(uppers) - min(lowers)
    print('Общая высота ледника Бара Шигри:', total_height)

Общая высота ледника Бара Шигри: 2600.0


In [129]:
total_area = 0

with open(filename, 'r', encoding='utf-8') as file:
    header = file.readline().strip().split(',')

    # Finding the indexes of the required columns
    name_index = header.index('glacier_name')
    area_index = header.index('area')

    # Read another lines
    for line in file:
        parts = line.strip().split(',')

        # check that lines have bara shigri in names
        if 'bara shigri' in parts[name_index].lower():
            area_str = parts[area_index]

            # if is not empty, plus to another
            if area_str != '':
                total_area += float(area_str)
                
print('Общая площадь ледника Бара Шигри:', total_area)

Общая площадь ледника Бара Шигри: 131099801.0


## Все, белка покупает билеты на ✈️сейлс